In [1]:
import argparse
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,TrainingArguments,Seq2SeqTrainer,Seq2SeqTrainingArguments,EarlyStoppingCallback
import pandas as pd
import torch
from rouge_chinese import Rouge
import numpy as np
from peft import LoraConfig, TaskType, get_peft_model
from nltk.translate.bleu_score import sentence_bleu

d:\Anaconda\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_name = "Langboat/mengzi-t5-base"
tokenizer_name = "Langboat/mengzi-t5-base"

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model.enable_input_require_grads()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
# for name,value in model.named_parameters():
#     print(name)

In [5]:
# from peft import LoraConfig, TaskType, get_peft_model

In [6]:
# lora_config = LoraConfig(
#         r=8,
#         target_modules=".*\.2[23].*query_key_value",
#         lora_dropout=0.05,
#         task_type=TaskType.CAUSAL_LM
#     )

In [7]:
# model = get_peft_model(model,lora_config)
# model.print_trainable_parameters()

In [8]:
dataset = pd.read_excel("name_remark_description.xlsx")

In [9]:
dataset = Dataset.from_pandas(dataset)
def preprocess_func(example):
    inputs = tokenizer("\t".join([
        # "物品类型："+example["type"]
        # ,"子类型："+example["sub_type"],
        example["name"]
        ,example["remark"]
    ]),max_length=50,truncation=True)
    outputs = tokenizer(text_target=example["description"],max_length=100,truncation=True)
    inputs["labels"]=outputs["input_ids"]
    
    return inputs
dataset = dataset.map(preprocess_func,remove_columns=dataset.column_names)

Map:   0%|          | 0/969 [00:00<?, ? examples/s]

In [10]:
train_args = Seq2SeqTrainingArguments(
    output_dir="test_seq2seq",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    gradient_checkpointing=True,
    num_train_epochs=2,
    logging_steps=1,
    save_total_limit=1
)

In [11]:
trainer = Seq2SeqTrainer(model=model,args=train_args,train_dataset=dataset,
                data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
                tokenizer=tokenizer)

In [12]:
trainer.train()

  0%|          | 0/90 [00:00<?, ?it/s]

d:\Anaconda\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 13.5978, 'learning_rate': 4.9444444444444446e-05, 'epoch': 0.03}
{'loss': 8.8957, 'learning_rate': 4.888888888888889e-05, 'epoch': 0.07}
{'loss': 7.0244, 'learning_rate': 4.8333333333333334e-05, 'epoch': 0.1}
{'loss': 6.4762, 'learning_rate': 4.7777777777777784e-05, 'epoch': 0.13}
{'loss': 6.221, 'learning_rate': 4.722222222222222e-05, 'epoch': 0.17}
{'loss': 6.0102, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.2}
{'loss': 5.7734, 'learning_rate': 4.6111111111111115e-05, 'epoch': 0.23}
{'loss': 5.7213, 'learning_rate': 4.555555555555556e-05, 'epoch': 0.26}
{'loss': 5.4071, 'learning_rate': 4.5e-05, 'epoch': 0.3}
{'loss': 5.3463, 'learning_rate': 4.4444444444444447e-05, 'epoch': 0.33}
{'loss': 5.215, 'learning_rate': 4.388888888888889e-05, 'epoch': 0.36}
{'loss': 5.1312, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.4}
{'loss': 5.1914, 'learning_rate': 4.277777777777778e-05, 'epoch': 0.43}
{'loss': 5.0871, 'learning_rate': 4.222222222222222e-05, 'epoch': 0.46}
{'los

TrainOutput(global_step=90, training_loss=4.460980206065708, metrics={'train_runtime': 391.1072, 'train_samples_per_second': 7.433, 'train_steps_per_second': 0.23, 'train_loss': 4.460980206065708, 'epoch': 2.97})

In [18]:
model = model.cuda()
ipt = tokenizer(
    #"物品类型：{}\n子类型：{}\n
    "{}\t{}".format("钢铁庇佑戒指","提升物理属性减伤率").strip()
    ,return_tensors="pt").to(model.device)

In [19]:
tokenizer.decode(model.generate(**ipt,max_length=100)[0], skip_special_tokens=True)

'圣职骑士的圣职, 可提升物理属性减伤率。 可提升物理属性减伤率。 据说是为骑士们所赠的戒指。'